# Example: Varying Injection Volume (Linearity)

Here outlines a basic linearity experiment where one sample (and therefore one vial location) is injected at varying injection volumes. 
* Varies injection volume within a range in a defined increment
* Samples in sample set defined in for loop, varying the injection volume

Get API Address

In [1]:
import os
from dotenv import load_dotenv

# get api address from .env file
load_dotenv("../.env_vars")
EMPOWER_API_ADDRESS = os.getenv("EMPOWER_API_ADDRESS")

# Import

In [1]:
from OptiHPLCHandler import EmpowerHandler

handler = EmpowerHandler(
    project="WebAPI_test",
    address=EMPOWER_API_ADDRESS,
)

# Configuration of methods, nodes, systems and plates

Using get attributes to: obtain method from list_method, node from list_nodes, system from list_systems within defined node and define the plate type from list_plates.

In [2]:
with handler:
    list_methods = handler.GetMethodList()
    method = list_methods[-1]  # most recent method
    print(f"Method Set Name: {method}")

    list_nodes = handler.GetNodeNames()
    node = list_nodes[40]
    print(f"Node: {node}")

    list_systems = handler.GetSystemNames(node=node)
    system = list_systems[0]
    print(f"System Configuration Name: {system}")

    list_plates = handler.GetPlateTypeNames()[:6]
    plate = list_plates[5]
    print(f"Plate: {plate}")
    plates = {"1": plate, "2": plate}

C:\Users\srfu\MATLAB\Projects\OptiHPLCHandler\src\OptiHPLCHandler\empower_api_core.py:250: UserWarning: The password will be sent in plain text.
  warnings.warn("The password will be sent in plain text.")


Method Set Name: CMS_test_23460013_2
Node: Node_name
System Configuration Name: 7337_HPLC1
Plate: ANSI-48Vial2mLHolder


Vary injection volume from a range of 1 to 3 µL in 1 µL increments

In [3]:
# Varying injection volume
sample_set_method_name = "test_samplesetmethod"

sample_list = []
list_inj = [i for i in range(1, 4, 1)]
# run linearity experiment from 1 to 3 µL in 1 µL increment

for i in range(len(list_inj)):
    sample_list.append(
        {
            "Method": method,
            "SamplePos": "1:A,1",  # specific vial selected
            "SampleName": f"Sample",
            "InjectionVolume": list_inj[i],
        },
    )

sample_list

[{'Method': 'CMS_test_23460013_2',
  'SamplePos': '1:A,1',
  'SampleName': 'Sample',
  'InjectionVolume': 1},
 {'Method': 'CMS_test_23460013_2',
  'SamplePos': '1:A,1',
  'SampleName': 'Sample',
  'InjectionVolume': 2},
 {'Method': 'CMS_test_23460013_2',
  'SamplePos': '1:A,1',
  'SampleName': 'Sample',
  'InjectionVolume': 3}]

Add column conditioning and equilibration at start, thus selecting column position.

In [4]:
conditioning = {
    "Function": {"member": "Condition Column"},
    "Method": method,
    "RunTime": 10,
    "ColumnPosition": {"member": "Position 3"},
}

equilibration = {
    "Function": {"member": "Equilibrate"},
    "Method": method,
    "RunTime": 7,
    "ColumnPosition": {"member": "Position 3"},
}

sample_list.insert(0, conditioning)
sample_list.insert(1, equilibration)
sample_list

[{'Function': {'member': 'Condition Column'},
  'Method': 'CMS_test_23460013_2',
  'RunTime': 10,
  'ColumnPosition': {'member': 'Position 3'}},
 {'Function': {'member': 'Equilibrate'},
  'Method': 'CMS_test_23460013_2',
  'RunTime': 7,
  'ColumnPosition': {'member': 'Position 3'}},
 {'Method': 'CMS_test_23460013_2',
  'SamplePos': '1:A,1',
  'SampleName': 'Sample',
  'InjectionVolume': 1},
 {'Method': 'CMS_test_23460013_2',
  'SamplePos': '1:A,1',
  'SampleName': 'Sample',
  'InjectionVolume': 2},
 {'Method': 'CMS_test_23460013_2',
  'SamplePos': '1:A,1',
  'SampleName': 'Sample',
  'InjectionVolume': 3}]

Post the experiment sample set method

In [5]:
with handler:
    handler.PostExperiment(
        sample_set_method_name=sample_set_method_name,
        sample_list=sample_list,
        plates=plates,
        audit_trail_message=None,
    )

HTTPError: HTTP error 409 with message 'The sample set method 'test_samplesetmethod' is already existing.' and ID 8dbef27fab44e35

Run the sample set method

In [ ]:
with handler:
    handler.RunExperiment(
        sample_set_method=sample_set_method_name,
        sample_set_name=sample_set_method_name,
        system=system,
        node=node,
    )